In [ ]:
!pip install langchain chromadb sentence-transformers langchain_groq numpy scikit-learn langchain_chroma langchain_huggingface groq

In [2]:
from google.colab import userdata
import os
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [18]:
from groq import Groq
client = Groq(api_key=os.environ.get('GROQ_API_KEY'))

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
embedder = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={'device':'cpu'}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
from langchain.document_loaders import WebBaseLoader
urls = [
    'https://python.langchain.com/docs/introduction/',
    'https://python.langchain.com/docs/concepts/#retrieval',
    'https://python.langchain.com/docs/concepts/#vector-stores'
]

all_docs = []
for url in urls:
    loader = WebBaseLoader(url)
    docs = loader.load()
    all_docs.extend(docs)

len(all_docs)

3

In [8]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(
    documents=all_docs,
    embedding=embedder,
    persist_directory="./chroma_db"
)

In [9]:
query = "What is retrieval-augmented generation and how does it work?"
query_embedding = embedder.embed_query(query)
raw_results = vectorstore.similarity_search_with_score(query, k=8)

print(f"🔍 Query: {query}")
print(f"📄 Retrieved {len(raw_results)} documents")

🔍 Query: What is retrieval-augmented generation and how does it work?
📄 Retrieved 3 documents


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

scored_results = []

for doc, similarity_score in raw_results:
    doc_embedding = embedder.embed_query(doc.page_content)

    cosine_sim = cosine_similarity([query_embedding], [doc_embedding])[0][0]

    query_words = set(query.lower().split())
    doc_words = set(doc.page_content.lower().split())
    keyword_overlap = len(query_words.intersection(doc_words)) / len(query_words) if query_words else 0

    doc_length = len(doc.page_content.split())
    query_length = len(query.split())
    length_ratio = min(doc_length / max(query_length * 3, 10), 1.0)

    confidence = (
        cosine_sim * 0.7 +
        keyword_overlap * 0.2 +
        length_ratio * 0.1
    )

    scored_results.append({
        'document': doc,
        'similarity_score': similarity_score,
        'confidence_score': confidence,
        'content': doc.page_content
    })

In [11]:
confidence_threshold = 0.4

filtered_results = [r for r in scored_results if r['confidence_score'] >= confidence_threshold]

filtered_results.sort(key=lambda x: x['confidence_score'], reverse=True)

print(f"🎯 Filtered to {len(filtered_results)} high-confidence documents")
print(f"📈 Confidence threshold: {confidence_threshold}")

🎯 Filtered to 3 high-confidence documents
📈 Confidence threshold: 0.4


In [13]:
print("📊 Retrieved Documents with Confidence Scores:")
print("=" * 60)

for i, result in enumerate(filtered_results[:5]):
    confidence = result['confidence_score']
    content = result['content']

    print(f"\n📄 Document {i+1}:")
    print(f"   🎯 Confidence: {confidence:.3f}")
    print(f"   📝 Content: {content[:200]}...")

    # Quality indicator
    if confidence > 0.6:
        print("   ✅ HIGH QUALITY")
    elif confidence > 0.4:
        print("   🟡 MEDIUM QUALITY")
    else:
        print("   ❌ LOW QUALITY")

📊 Retrieved Documents with Confidence Scores:

📄 Document 1:
   🎯 Confidence: 0.506
   📝 Content: 




Conceptual guide | ü¶úÔ∏èüîó LangChain








Skip to main contentOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.IntegrationsAPI ReferenceMoreContri...
   🟡 MEDIUM QUALITY

📄 Document 2:
   🎯 Confidence: 0.506
   📝 Content: 




Conceptual guide | ü¶úÔ∏èüîó LangChain








Skip to main contentOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.IntegrationsAPI ReferenceMoreContri...
   🟡 MEDIUM QUALITY

📄 Document 3:
   🎯 Confidence: 0.488
   📝 Content: 




Introduction | ü¶úÔ∏èüîó LangChain








Skip to main contentOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.IntegrationsAPI ReferenceMoreContributi...
   🟡 MEDIUM QUALITY


In [25]:
high_conf_docs = [r for r in filtered_results if r['confidence_score'] > 0.5]
context = "\n\n".join([doc['content'] for doc in high_conf_docs[:3]])

prompt = f"""Based on the following high-confidence context, answer the user's question accurately.

Context:
{context}

Question: {query}

Answer:"""

try:
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=500
    )

    answer = response.choices[0].message.content

    print("🤖 Generated Answer:")
    print("=" * 40)
    print(answer)

except Exception as e:
    print(f"❌ Error: {str(e)}")

🤖 Generated Answer:
Retrieval-augmented generation (RAG) is a technique that enhances language models by combining them with external knowledge bases. It works by using a retriever component to return relevant documents from a knowledge base in response to a query, and then using a language model to generate text based on the retrieved documents. This approach allows the language model to access a large amount of external knowledge and generate more accurate and informative text. In the context of LangChain, RAG is a key concept that enables the creation of more sophisticated AI applications that can retrieve and generate text based on external knowledge.


In [29]:
high_conf = [r for r in scored_results if r['confidence_score'] > 0.6]
medium_conf = [r for r in scored_results if 0.3 <= r['confidence_score'] <= 0.6]
low_conf = [r for r in scored_results if r['confidence_score'] < 0.3]

print(f"📊 Confidence Distribution Analysis:")
print(f"🟢 High Confidence (>0.6): {len(high_conf)} documents")
print(f"🟡 Medium Confidence (0.3-0.6): {len(medium_conf)} documents")
print(f"🔴 Low Confidence (<0.3): {len(low_conf)} documents")

if high_conf:
    print(f"\n🏆 Highest Confidence: {high_conf[0]['confidence_score']:.3f}")
    print(f"📝 Best Match: {high_conf['content'][:150]}...")

📊 Confidence Distribution Analysis:
🟢 High Confidence (>0.6): 0 documents
🟡 Medium Confidence (0.3-0.6): 3 documents
🔴 Low Confidence (<0.3): 0 documents
